In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Aqui estão as regras de validação para cada campo, sem justificativas, e as regras de `append` também incluídas:

### Regras de Validação

1. **PERMISSÃO**:
   - Deve ser `"PERMISSÃO"` ou estar vazio.

2. **ACC**:
   - Deve ser `"AUTORIZADO"` ou estar vazio.

3. **VÁLIDA EM TODO**:
   - Deve ser um número (`isdigit()`) e `width < height`.

4. **Nº REGISTRO**:
   - Deve ser um número (`isdigit()`) e `width > height`.

5. **CPF**:
   - Deve seguir o formato `xxx.xxx.xxx-xx` (regex: `\d{3}\.\d{3}\.\d{3}-\d{2}`).

6. **CAT. HAB.**:
   - Deve ter no máximo duas letras e não pode estar na lista de estados brasileiros.

7. **1ª HABILITAÇÃO, DATA NASCIMENTO, VALIDADE**:
   - Deve conter uma data no formato `dd/mm/aaaa` (regex: `\d{2}/\d{2}/\d{4}`).

8. **DOC. IDENTIDADE/ÓRG EMISSOR/UF**:
   - Deve ser um número, ou duas letras de um estado brasileiro, ou ter 3 ou 4 caracteres ou o valor deve contar letras e números.

9. **FILIAÇÃO**:
   - Deve conter apenas letras, exceto a palavra `"PERMISSÃO"`, e ter mais de 3 caracteres.

### Regras de Append

1. **FILIAÇÃO**:
   - Todas as transcrições válidas são adicionadas.

2. **DOC. IDENTIDADE/ÓRG EMISSOR UF** ou **DOC. IDENTIDADE/ÓRG EMISSOR/UF**:
   - Se o valor contiver letras e números, o candidato mais próximo é adicionado.
   - Caso contrário, até 3 transcrições válidas são adicionadas.

3. **Outros Candidatos**:
   - O candidato válido mais próximo é adicionado, se não estiver duplicado.

In [2]:
import math
import os
import re

lista_rotulos = [
    'NOME', 'CPF', 'PERMISSÃO', 'ACC', 'CAT. HAB.',
    '1ª HABILITAÇÃO', 'VALIDADE', 'Nº REGISTRO', 'VÁLIDA EM TODO', 'DATA NASCIMENTO',
    'DOC. IDENTIDADE/ÓRG EMISSOR/UF', 'DOC. IDENTIDADE/ÓRG EMISSOR UF', 'DOC IDENTIDADE/ÓRG EMISSOR/UF', 'FILIAÇÃO'
]

estados_brasileiros = [
    "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA",
    "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN",
    "RS", "RO", "RR", "SC", "SP", "SE", "TO"
]

def converter_para_tuplas(conteudo):
    boxes = []
    linhas = conteudo.strip().split('\n')

    for linha in linhas[1:]:
        valores = linha.split(', ')

        # Verifica se a linha começa com '[', indicando o formato alternativo
        if linha.startswith('['):
            partes = linha.replace('[', '').replace(']', '').split(', ')
            x = int(partes[0])
            y = int(partes[1])
            width = int(partes[2]) - x
            height = int(partes[3]) - y
            transcription = partes[-1]
            boxes.append((x, y, width, height, transcription))
        else:
            # Formato original com 5 valores
            if len(valores) == 5:
                x = int(valores[0])
                y = int(valores[1])
                width = int(valores[2])
                height = int(valores[3])
                transcription = valores[4]
                boxes.append((x, y, width, height, transcription))

    return boxes


def validar_transcription(rotulo, transcription, width, height):
    if rotulo == 'PERMISSÃO':
        return transcription == "PERMISSÃO"
    elif rotulo == 'ACC':
        return transcription == "AUTORIZADO"
    elif rotulo == 'VÁLIDA EM TODO':
        return transcription.isdigit() and width < height
    elif rotulo == 'Nº REGISTRO':
        return transcription.isdigit() and width > height
    elif rotulo == 'CPF':
        return bool(re.match(r'\d{3}\.\d{3}\.\d{3}-\d{2}', transcription))
    elif rotulo == 'CAT. HAB.':
        return len(transcription) <= 2 and transcription not in estados_brasileiros
    elif rotulo in ['1ª HABILITAÇÃO', 'DATA NASCIMENTO', 'VALIDADE']:
        return bool(re.match(r'\d{2}/\d{2}/\d{4}', transcription))
    elif rotulo in ['DOC. IDENTIDADE/ÓRG EMISSOR UF', 'DOC. IDENTIDADE/ÓRG EMISSOR/UF']:
        return (
            transcription.isdigit() or
            (len(transcription) == 2 and transcription in estados_brasileiros) or
            len(transcription) == 3 or
            len(transcription) == 4 or
            (any(c.isalpha() for c in transcription) and any(c.isdigit() for c in transcription) and all(c.isalnum() or c.isspace() for c in transcription))
        )
    elif rotulo == 'FILIAÇÃO':
        return transcription.replace(" ", "").isalpha() and "PERMISSÃO" not in transcription and len(transcription) > 3
    return True

def calcular_distancia(box1, box2):
    x1, y1 = box1[0], box1[1]
    x2, y2 = box2[0], box2[1]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def ordenar_por_rotulo(lista_rotulos, boxes):
    resultado = []
    transcricoes_adicionadas = set()
    rotulos_adicionados = set()
    data_nascimento = None
    primeira_valida_em_todo = None
    segunda_valida_em_todo = None
    valida_em_todo_boxes = [box for box in boxes if box[4] == 'VÁLIDA EM TODO']
    validade_box = next((box for box in boxes if box[4] == 'VALIDADE'), None)
    valor_espelho = None

    if len(valida_em_todo_boxes) == 2 and not validade_box:
        primeira_valida_em_todo, segunda_valida_em_todo = sorted(valida_em_todo_boxes, key=lambda box: box[1])

    for rotulo in lista_rotulos:
        box_rotulo = next((box for box in boxes if box[4] == rotulo and rotulo not in rotulos_adicionados), None)
        if box_rotulo:
            rotulos_adicionados.add(rotulo)

            candidatos_numero = []
            for box in boxes:
                if re.match(r'^\d{9}$', box[4]) and box[4] not in transcricoes_adicionadas and (box[0] < box[1] or box[0] == -1):
                    candidatos_numero.append(box)

            if candidatos_numero:
                valor_espelho = candidatos_numero[0]

            if rotulo == 'DATA NASCIMENTO':
                data_nascimento = box_rotulo

            if rotulo == 'VÁLIDA EM TODO':
                if valor_espelho:
                   resultado.append(box_rotulo)
                   resultado.append(valor_espelho)
                   transcricoes_adicionadas.add(valor_espelho[4])

                elif candidatos_numero and not segunda_valida_em_todo:
                    candidato_proximo_numero = min(candidatos_numero, key=lambda box: calcular_distancia(box_rotulo, box))
                    resultado.append(candidato_proximo_numero)
                    transcricoes_adicionadas.add(candidato_proximo_numero[4])

            if not validade_box and primeira_valida_em_todo and segunda_valida_em_todo and rotulo == 'VÁLIDA EM TODO':
                candidatos_data = [
                    box for box in boxes
                    if re.match(r'\d{2}/\d{2}/\d{4}', box[4]) and box[4] not in transcricoes_adicionadas
                ]
                if candidatos_data:
                    candidato_proximo_data = min(candidatos_data, key=lambda box: calcular_distancia(primeira_valida_em_todo, box))
                    resultado.append((primeira_valida_em_todo[0], primeira_valida_em_todo[1], primeira_valida_em_todo[2], primeira_valida_em_todo[3], 'VALIDADE'))
                    resultado.append(candidato_proximo_data)
                    transcricoes_adicionadas.add('VALIDADE')

            elif rotulo == 'PERMISSÃO':
                candidatos = [box for box in boxes if box[4] == "PERMISSÃO" and box[4] not in transcricoes_adicionadas]
                if len(candidatos) == 2:
                    candidatos_ordenados = sorted(candidatos, key=lambda box: box[1])
                    resultado.extend(candidatos_ordenados)
                    transcricoes_adicionadas.update(box[4] for box in candidatos_ordenados)
            else:
                candidatos = [box for box in boxes if box[4] not in lista_rotulos and box[4] not in transcricoes_adicionadas]
                if candidatos and box_rotulo not in transcricoes_adicionadas:
                    candidatos_validos = [box for box in candidatos if validar_transcription(rotulo, box[4], box[2], box[3])]

                    if rotulo == 'FILIAÇÃO' and candidatos_validos:
                        resultado.append(box_rotulo)
                        resultado.extend(candidatos_validos)
                    elif rotulo in ['DOC. IDENTIDADE/ÓRG EMISSOR UF', 'DOC. IDENTIDADE/ÓRG EMISSOR/UF', 'DOC IDENTIDADE/ÓRG EMISSOR/UF'] and candidatos_validos:
                        adicionou_linha = False
                        for candidato in candidatos_validos:
                              if any(c.isalpha() for c in candidato[4]) and any(c.isdigit() for c in candidato[4]):
                                  candidatos_validos = [box for box in candidatos_validos if box[4] not in transcricoes_adicionadas]
                                  if candidatos_validos and not adicionou_linha:
                                      candidato_proximo = min(candidatos_validos, key=lambda box: calcular_distancia(box_rotulo, box))
                                      resultado.append(box_rotulo)
                                      resultado.append(candidato_proximo)
                                      transcricoes_adicionadas.add(candidato_proximo[4])
                                      adicionou_linha = True
                                      break
                        if not adicionou_linha:
                              candidatos_validos = [box for box in candidatos_validos if box[4] not in transcricoes_adicionadas][:3]
                              resultado.append(box_rotulo)
                              resultado.extend(candidatos_validos)
                              transcricoes_adicionadas.update(box[4] for box in candidatos_validos)
                    elif candidatos_validos:
                        candidatos_validos = [box for box in candidatos_validos if box[4] not in transcricoes_adicionadas]
                        if candidatos_validos:
                            candidato_proximo = min(candidatos_validos, key=lambda box: calcular_distancia(box_rotulo, box))
                            resultado.append(box_rotulo)
                            resultado.append(candidato_proximo)
                            transcricoes_adicionadas.add(candidato_proximo[4])

    if not resultado and data_nascimento:
        resultado.append(data_nascimento)

    return resultado

def salvar_ordenado_em_arquivo(novo_conteudo, nome_arquivo):
    boxes = converter_para_tuplas(novo_conteudo)
    resultado_ordenado = ordenar_por_rotulo(lista_rotulos, boxes)
    pasta_nova = '/content/drive/MyDrive/novos_gt'
    if not os.path.exists(pasta_nova):
        os.makedirs(pasta_nova)

    nome_arquivo_saida = os.path.join(pasta_nova, f'{os.path.splitext(os.path.basename(nome_arquivo))[0]}.txt')

    with open(nome_arquivo_saida, 'w') as f:
        for box in resultado_ordenado:
            x, y, width, height, transcription = box
            f.write(f"{x}, {y}, {width}, {height}, {transcription}\n")

    print(f"Arquivo salvo em: {nome_arquivo_saida}")

def processar_pasta():
    pasta = '/content/drive/MyDrive/BD/BID Dataset/CNH_Frente'
    # arquivos_txt = ['00006815_gt_ocr.txt']
    arquivos_txt = [f for f in os.listdir(pasta) if f.endswith('.txt')]

    for arquivo in arquivos_txt:
        caminho_arquivo = os.path.join(pasta, arquivo)

        with open(caminho_arquivo, 'r', encoding='latin1') as f:
            conteudo = f.readlines()

        ignore_list = [
            'REPÚBLICA FEDERATIVA DO BRASIL', 'MINISTÉRIO DAS CIDADES',
            'DEPARTAMENTO NACIONAL DE TRÂNSITO', 'CARTEIRA NACIONAL DE HABILITAÇÃO',
            'O TERRITÓRIO NACIONAL', 'REPÚBLICA FEDERATIVA DO BRA',
        ]
        linhas_filtradas = [linha for linha in conteudo if not any(ignorar in linha for ignorar in ignore_list)]
        novo_conteudo = ''.join(linhas_filtradas)

        salvar_ordenado_em_arquivo(novo_conteudo, caminho_arquivo)

processar_pasta()

Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006862_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006865_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006863_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006866_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006877_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006883_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006875_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006878_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006882_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006879_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006880_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006873_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006876_gt_ocr.txt
Arquivo salvo em: /content/drive/MyDrive/novos_gt/00006881_gt_ocr.txt
Arquivo salvo em: /c

In [ ]:
import os

# Definindo a pasta
pasta_txt = '/content/drive/MyDrive/novos_gt'

# Definindo a lista de rótulos
lista_rotulos = [
    'NOME', 'DATA NASCIMENTO', 'CPF', 'PERMISSÃO', 'ACC', 'CAT. HAB.',
    '1ª HABILITAÇÃO', 'VALIDADE', 'Nº REGISTRO', 'VÁLIDA EM TODO',
    'DOC. IDENTIDADE/ÓRG EMISSOR/UF', 'FILIAÇÃO'
]

# Listas para armazenar arquivos faltantes e presentes de cada rótulo
faltantes = {rotulo: [] for rotulo in lista_rotulos}
presentes = {rotulo: [] for rotulo in lista_rotulos}

# Função para verificar os rótulos no arquivo, lendo a quinta coluna
def verificar_rotulos(arquivo_txt):
    rotulos_encontrados = set()

    with open(arquivo_txt, 'r', encoding='utf-8') as f:
        for linha in f:
            partes = linha.strip().split(',')
            if len(partes) == 5:
                rotulo = partes[4].strip()
                # Adicionando o rótulo encontrado
                rotulos_encontrados.add(rotulo)

    return rotulos_encontrados

# Função para extrair o código do arquivo (parte antes de '_gt_ocr.txt')
def extrair_codigo(arquivo):
    if arquivo.endswith('_gt_ocr.txt'):
        return arquivo.split('_')[0]
    return None

# Iterando sobre os arquivos txt na pasta
for arquivo in os.listdir(pasta_txt):
    if arquivo.endswith('.txt'):
        caminho_arquivo = os.path.join(pasta_txt, arquivo)
        rotulos_encontrados = verificar_rotulos(caminho_arquivo)
        codigo_arquivo = extrair_codigo(arquivo)

        # Verificando rótulos faltantes e presentes
        for rotulo in lista_rotulos:
            if rotulo == 'DOC. IDENTIDADE/ÓRG EMISSOR/UF':
                # Verificando os dois casos de 'DOC. IDENTIDADE'
                if 'DOC. IDENTIDADE/ÓRG EMISSOR/UF' in rotulos_encontrados or 'DOC. IDENTIDADE/ÓRG EMISSOR UF' in rotulos_encontrados or 'DOC IDENTIDADE/ÓRG EMISSOR/UF' in rotulos_encontrados:
                    presentes[rotulo].append(codigo_arquivo)
                else:
                    faltantes[rotulo].append(codigo_arquivo)
            else:
                if rotulo in rotulos_encontrados:
                    presentes[rotulo].append(codigo_arquivo)
                else:
                    faltantes[rotulo].append(codigo_arquivo)

# Printando o total de arquivos faltantes e presentes para cada rótulo
for rotulo in lista_rotulos:
    print(f"Rótulo '{rotulo}':")
    print(f"  Total de arquivos que têm o rótulo: {len(presentes[rotulo])}")
    print(f"    Códigos dos arquivos: {presentes[rotulo] if presentes[rotulo] else 'Nenhum arquivo contém este rótulo.'}")
    print(f"  Total de arquivos que estão faltando o rótulo: {len(faltantes[rotulo])}")
    print(f"    Códigos dos arquivos: {faltantes[rotulo] if faltantes[rotulo] else 'Nenhum arquivo está faltando este rótulo.'}")
    print("-" * 50)


Rótulo 'NOME':
  Total de arquivos que têm o rótulo: 3600
    Códigos dos arquivos: ['00006815', '00006862', '00006865', '00006863', '00006866', '00006877', '00006883', '00006875', '00006878', '00006882', '00006879', '00006880', '00006873', '00006876', '00006881', '00006874', '00006884', '00006891', '00006890', '00006892', '00006896', '00006894', '00006885', '00006888', '00006887', '00006895', '00006893', '00006889', '00006886', '00006902', '00006907', '00006904', '00006899', '00006897', '00006901', '00006898', '00006906', '00006900', '00006903', '00006905', '00006918', '00006916', '00006915', '00006911', '00006909', '00006913', '00006908', '00006914', '00006912', '00006917', '00006910', '00006919', '00006926', '00006927', '00006920', '00006925', '00006924', '00006923', '00006922', '00006929', '00006921', '00006928', '00006932', '00006933', '00006939', '00006935', '00006937', '00006931', '00006930', '00006936', '00006934', '00006938', '00006941', '00006940', '00006947', '00006945', '00